In [ ]:
import pmcx
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
res = pmcx.run(nphoton=1000000, 
               vol=np.ones([60, 60, 60], dtype='uint8'), 
               tstart=0, 
               tend=5e-9, 
               tstep=5e-9, 
               srcpos=[30,30,0], 
               srcdir=[0,0,1], 
               prop=np.array([[0, 0, 1, 1], [0.005, 1, 0.01, 1.37]]),
               )
res['flux'].shape

plt.imshow(np.log10(res['flux'][30,:, :]))
plt.show()

In [ ]:
def integral_klein_nishina(theta, wavelen):
    # Adapted from https://physics.stackexchange.com/questions/690255/angle-energy-dependent-cdf-for-compton-scattering

    cosTheta = np.cos(theta)

    h = 6.62607015e-34
    c = 299792458
    m = 9.10938356e-31

    R = (h*wavelen)/(m*c**2)

    out = -(cosTheta/R**2) + np.log(1 + R*(1-cosTheta)) * (1/R - 2/R**2 - 2/R**3)
    out = out - 1/(2*R*(1+R*(1-cosTheta))**2) + 1/(1+R*(1-cosTheta)) * (-2/R**2 - 1/R**3)

    return out

def cdf_klein_nishina(theta, wavelen):
    assert theta >= 0 and theta <= np.pi
    out = (integral_klein_nishina(theta, wavelen) - integral_klein_nishina(0, wavelen))
    out = out / (integral_klein_nishina(np.pi, wavelen) - integral_klein_nishina(0, wavelen))
    return out